# 06 - Autoencoder Clássico (Self-supervised Learning)

## Importação

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics import median_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline

## Carga dos Dados

In [ ]:
train = pd.read_csv("data-processed/train.csv")
val = pd.read_csv("data-processed/val.csv")

train['DATE_TIME'] = pd.to_datetime(train['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')
val['DATE_TIME'] = pd.to_datetime(val['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
train2 = train.copy()
train2['WEEKDAY'] = train2['DATE_TIME'].dt.weekday
train2['HOUR'] = train2['DATE_TIME'].dt.hour
train2['MINUTE'] = train2['DATE_TIME'].dt.minute

val2 = val.copy()
val2['WEEKDAY'] = val2['DATE_TIME'].dt.weekday
val2['HOUR'] = val2['DATE_TIME'].dt.hour
val2['MINUTE'] = val2['DATE_TIME'].dt.minute

## Autoencoder

In [ ]:
encoder = OneHotEncoder(cols=cats, use_cat_names=True)
train2_ohe = encoder.fit_transform(train2[cats])
val2_ohe = encoder.transform(val2[cats])

import torch.nn as nn
import torch
import torch.optim as opt
import torch.nn.functional as F

train2_ohe.shape

In [ ]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()

        self.hid1 = nn.Linear(57, 100)
        self.hid2 = nn.Linear(100, 20)
        self.hid3 = nn.Linear(20, 100)
        self.out = nn.Linear(100, 57)
    
    def forward(self, x, return_embeddings=False):
        x = F.relu(self.hid1(x))
        x = self.hid2(x)
        
        if return_embeddings:
            return x
        
        x = F.relu(self.hid3(x))
        x = self.out(x)

        return x

In [ ]:
torch.manual_seed(0)
net = AE().cuda()
criterion = nn.MSELoss()
optimizer = opt.Adam(net.parameters(), lr=1e-4)

x = torch.from_numpy(train2_ohe.values).cuda().float()
for e in range(10000):
    optimizer.zero_grad()
    
    p = net(x).cuda()
    loss = criterion(p, x)
    #print(loss.item())
    
    loss.backward()
    optimizer.step()
    
    p_numpy = p.cpu().detach().numpy()
    if e % 500 == 0:
        print(np.sqrt(mean_squared_error(train2_ohe, p_numpy)))

## Embedding to Tree

In [ ]:
from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

x = torch.from_numpy(train2_ohe.values).cuda().float()
Xtr = net(x,return_embeddings=True).detach().cpu().numpy()
ytr = train2['Y4WIN']
mdl.fit(Xtr,ytr)

x = torch.from_numpy(val2_ohe.values).cuda().float()
Xval = net(x,return_embeddings=True).detach().cpu().numpy()

p = mdl.predict(Xval)
median_absolute_error(val2['Y4'], p)

In [ ]:
Xtr

# Fim